In [ ]:
from assignment2 import *
pd.options.display.float_format = '{:,.4f}'.format

Remise électronique sur ZoneCours au plus tard le **14 avril à 23h55** de:

* Ce Jupyter notebook (.ipynb) en guise de rapport, une fois les réponses ajoutées. 
* **Tous** les fichiers *.py* nécessaires pour l'exécution du notebook

Le rapport **doit**:

* être en format .ipynb (des points seront enlevés pour tout autre format).
* inclure le numéro de matricule de tous les membres de l'équipe dans la première cellule (i.e. remplacez ces consignes)
* répondre aux questions et discuter des résultats à l'aide de tables, graphiques, et cellules markdown

Barème:

* **35%** Exactitude des résultats
* **25%** Discussions complètes et **concises** (cellules ``markdown'')
* **20%** Concision du notebook (.ipynb avec minimum de code) et clarté des tables & graphiques
* **20%** Clarté du code (.py)

Dans les tables, présentez 4 décimales après le point.

# Question 1

Nous considérons des données historiques d'options sur le S&P 500. Le fichier suivant sont dans le répertoire courant:
```
215K 28 Mar 20:48 distrd_108105.csv
2.7M 28 Mar 21:01 opprcd200201_108105.csv
 63K 28 Mar 21:02 opprcd961203_108105.csv
511K 28 Mar 20:45 secprd_108105.csv
9.3M 28 Mar 20:43 zerocd.csv
```

Les `secprd_108105.csv` et `zerocd.csv` contiennent, respectivement, les données sur le sous-jacent et la courbe des taux sans risque (capitalisés continuement). La fonction `get_log_excess_returns` (fournie dans `assignment2.py`). 

In [ ]:
days_in_year = 252
log_xreturns = get_log_excess_returns(days_in_year)

Utilisez les paramètres initiaux suivants pour démarrer l'estimation des paramètres d'un modèle NGARCH en date du 1996-12-31. Utilisez la notation suivante:

\begin{align*}
r_{t+1} - r_{f} &= \lambda \sqrt{h_{t+1}}-\frac{1}{2}h_{t+1}+\sqrt{h_{t+1}}%
\varepsilon _{t+1}, \\
h_{t+1}& =\omega +\alpha h_{t}\left( \varepsilon _{t}-\gamma \right)
^{2}+\beta h_{t}
\end{align*}

Quels paramètres obtenez vous en maximisant la vraisemblance des rendements au 1996-12-31? Répétez l'exercice au 2020-02-01. Tabulez les paramètres de façon à pouvoir comparer les deux jeux de paramètres aisément. Ajoutez à votre table 

- la volatilité inconditionnelle (annualisée)
- la volatilité conditionnelle (annualisée) prédite au lendemain de l'estimation
- la correlation conditionnelle en $t$ entre $r_{t+1}$ et $h_{t+2}$
- la volatilité conditionnelle en $t$ de la variance $h_{t+2}$

que les paramètres impliques. De plus, inspirez-vous de la `note_on_erp.ipynb` et définissez une fonction:
```
def plot_var_forecasts(horizon, P, Q, annualized=False):
```
que vous appelerez avec `annualized=True` sur des simulations résultant des deux différents jeux de paramètres. Discutez des paramètres, des figures et des implications aux point de vue de la validité de la spécification (vs *misspecification*) du NGARCH.

In [ ]:
# Inputs pour initialiser le MLE
time_t = np.datetime64('1996-12-31')
ng1996 = ngarch.initialize_at(time_t, log_xreturns, days_in_year)

time_t = np.datetime64('2020-02-01')
ng2020 = ngarch.initialize_at(time_t, log_xreturns, days_in_year)

# MLE + nouveaux paramètres NGARCH
ng1996 = f_NGARCH(ng1996)
ng2020 = f_NGARCH(ng2020)

# Table de sortie
out_Q1 = f_out_format_Q1([ng1996, ng2020])
out_Q1

In [ ]:
# Inputs pour les simulations 
n_days  = 10*days_in_year
n_paths = 10000 

# Simulation sous P
P_1996 = measure( *ng1996.simulateP(100, n_days, n_paths, ng1996.P_predict_h()))
P_2020 = measure( *ng2020.simulateP(100, n_days, n_paths, ng2020.P_predict_h(), P_1996.z))

# Simulation sous Q
Q_1996 = measure( *ng1996.simulateQ(100, n_days, n_paths, ng1996.Q_predict_h(), P_1996.z))
Q_2020 = measure( *ng2020.simulateQ(100, n_days, n_paths, ng2020.Q_predict_h(), P_1996.z))

# Graphique des variances conditionnelles prévues
horizon = np.arange(1,n_days+2) / days_in_year
plot    = plot_var_forecasts(horizon = horizon, P = [P_1996, P_2020], Q = [Q_1996, Q_2020], annualized=True)

NOTE : Variance plus grande sous Q que sous P car on surpondère les évènements extrême. DOnc queux de distribution plus lourde et donc, variance plus lourde.

# Question 2

Nous souhaitons tarifer les options aux deux dates pour lesquelles nous avons estimé les paramètres ci-haut. Avant de procédez, comparer qualitativement les deux jeux de données. Soutenez votre propos à l'aide de statistiques descriptives (tables) et de figures au besoin. Quels filtres devrait-on appliquer aux données avant procéder à la tarification des options? Justifier.

In [ ]:
# Construction de la table descriptive
options96   = pd.read_csv('opprcd961203_108105.csv', index_col=0).reset_index(drop=True)
options20   = pd.read_csv('opprcd200201_108105.csv', index_col=0).reset_index(drop=True)
option_info = pd.concat([options96, options20])

f_describe_table(option_info).dropna(axis = 1)

La première étape, avant même de construire la table de statistiques descriptives, a été d'enlever les données pour lesquelles on a une volatilité implicite manquante et filtrer les lignes identiques. Ce sont les premiers filtres à appliquer aux données avant la tarification. Il est ensuite possible de comparer les deux jeux de données à l'aide de la table ci-dessus.

Le premier constat frappant est la différence entre le nombre d'option pour lesquels on a des données. En effet, il y en a plus de 43 fois plus en 2020 qu'en 1996. L'intervalle de prix d'exercice possible a aussi beaucoup augmenté. Par exemple, pour les puts, on voit que le prix d'exercice minimal est plus bas en 2020 qu'en 1996, et ce, malgré le fait que le prix du S&P500 a beaucoup augmenté depuis. Ce phénomène est aussi observable à travers l'augmentation des valeurs maximales de volatilité implicite entre 1996 et 2020. Finalement, il y a aussi un écart entre les maturités offertes. On peut observer que des options autant à plus courte qu'à plus longue échéance sont disponibles en 2020 par rapport à 1996.

Pour ce qui est des autres filtres à appliquer aux données avant la tarification, on doit premièrement s'assurer qu'il n'y a pas d'options avec une volatilité implicite extrême. En effet, une volatilité implicite extrême tend à ne pas représenter les vraies attentes des participants du marché. Nous fixerons à 125% la volatilité maximale pour laquelle on garde les données des options. Ensuite, on doit simplement s'assurer qu'il n'y a pas d'incohérence dans les données du delta (delta des calls entre 0 et 1 avec delta des puts entre -1 et 0). Le dernier filtre à appliquer sera sur le temps à maturité des options. Sachant que des options à très court terme peuvent se comporter de manière anormal, nous allons filtrer ceux avec une maturité de moins de 7 jours (une semaine). Nous allons aussi filtrer les options avec de très longue maturité car celles-ci manquent souvent de liquidité. La contrainte maximale pour la maturité sera de 550 jours (environ 1 an et demi).

Il reste maintenant à appliquer les filtrations décrites. Or, on s'apperçoit à l'aide de la table descriptive qu'il sera seulement nécessaire de filtrer les données de 2020 par rapport au temps restant avant la maturité. C'est ce qui est fait ci-dessous.


In [ ]:
# Filtration des données 
option_info = option_info[option_info['DTM'] > 7]
option_info = option_info[option_info['DTM'] < 550]
option_info = f_clean_table(option_info).reset_index(drop = True)

# Question 3

Afin de tarifer les options, nous devons calculer, pour chaque paire $(t,T)$, le prix ex-dividendes du sous-jacent. Dans le cas du S&P 500, deux options s'offre à nous:

1. Utiliser en $t$ le taux de dividende prévu ($y$, en termes annuels) par OptionMetrics pour calculer $S(t) e^{-y(T-t)}$.

2. Utiliser l'approche du Cboe pour calculer le option-implied forward price $F(t,T)$ et la courbe des taux fournis par OptionMetrics (`zerocd.csv`). Ensuite, calculer le ex-dividend price $F(t,T) e^{-r_{t,T} (T-t)}$, où $r_{t,T}$ est le taux sans risque à $t$ pour une maturité à $T$, 

Comparer les résultats obtenus à les des deux approches. Comment et pourquoi diffèrent-ils selon vous? 

In [ ]:
# Ajouter les prix, les dividende, les rf et la moneyness à option_info
f_add_Q3_info(option_info, days_in_year)

# Méthode 1 
exdiv_1 = option_info.S_t * np.exp(-option_info.y_t * (option_info.DTM)/days_in_year)
option_info['exdiv_1'] = exdiv_1

# Méthode 2
option_info = f_F_CBOE(option_info)
exdiv_2     = option_info.F_CBOE * np.exp(-option_info.r_f * (option_info.DTM)/days_in_year)
option_info['exdiv_2'] = exdiv_2


# Test (PAS ENCORE CERTAIN DU RÉSULTAT CAR LES 2 MÉTHODES DONNENT PRESQUE LA MÊME CHOSE)
date_unique = np.unique(option_info['date'])
dtm_unique = np.unique(option_info.loc[option_info.date == date_unique[1]].DTM)
ex1_unique = np.unique(option_info.loc[option_info.date == date_unique[1]].exdiv_1)
ex2_unique = np.unique(option_info.loc[option_info.date == date_unique[1]].exdiv_2)

pd.DataFrame({
    'DTM_2020' : dtm_unique,
    'EX1_2020' : ex1_unique,
    'EX2_2020' : ex2_unique,
    'DIFF'     : ex1_unique - ex2_unique,
    'DIFF_BOOl': (ex1_unique - ex2_unique) == 0
})




# Question 4

Notez que sous BMS, un call
\begin{align*}
 c(t,T,K) &= \hat{S}(t) N(d_1) - Ke^{-r_f(T-t)} N(d_2) \\
% \textrm{avec} 
 d_i &= \frac{\log\frac{\hat{S}(t)}{K} + \left(r_f \pm \frac{1}{2}\sigma^2 \right) (T-t)
     }{\sigma\sqrt{T-t}}
\end{align*}
avec $\hat{S}(t) = S(t)e^{-y(T-t)} = F(t,T)e^{-r_f(T-t)}$. L'argument est similaire pour un put. On peut donc calculer les volatilités implicites de chaque option avec l'approche utilisée dans le TP1, avec $y=0$, mais en utilisant les prix ex-dividendes obtenus en Question 3. 

Comparer les 2 volatilités implicites ainsi obtenues avec celle fournie par OptionMetrics (`impl_volatility`). Illustrer votre comparaison. 

In [ ]:
impl_vol_mkt = bms.implied_volatility(opt_price = option_info.mean_bidask, 
                                      S         = [option_info.exdiv_1,option_info.exdiv_2], 
                                      K         = option_info.strike_price / 1000, 
                                      r         = option_info.r_f, 
                                      y         = 0, 
                                      T         = option_info.DTM / days_in_year, 
                                      is_call   = (option_info.cp_flag == 'C'))

In [ ]:
option_info['IV_method1'] = impl_vol_mkt[0]
option_info['IV_method2'] = impl_vol_mkt[1]
option_info[['date','exdate','cp_flag', 'strike_price','impl_volatility','IV_method1','IV_method2']]

# Question 5

Dans la même optique, dénotons $\tilde{R}_{t,T}^{(j)} = \exp\left\lbrace\sum_{k=1}^{T-t} r_{t+k}^{(j)} - r_f\right\rbrace$ le rendement excédentaire à terme obtenus par simulation du modèle NGARCH. Notez que le payoff actualisé d'une option peut être écrit 
\begin{align*}
  \mathcal{O}^{(j)}_{i} = e^{-r_{t,T_i} (T_i-t)}\max\left\lbrace 0, cp_i\left(F_{t,T_i} \tilde{R}_{t,T_i}^{(j)} - K\right) \right\rbrace
\end{align*}
où $cp_i = 1$ pour un call $i$ ($cp_i = -1$ pour le put $i$) de maturité $T_i$ et de strike $K_i$. Ainsi, seul $\tilde{R}_{t,T}^{(j)}$ dépend de la trajectoire simulée. En $t$, en simulant une seule fois 100,000 trajectoires de longueur $T_{max} - t$, il est donc possible de tarifer toutes les options observées ce jour là.

Procéder ainsi en utilisant l'un où l'autre des prix forward considérés en Question 3. Comparer les IV obtenues en Question 4, selon les prix de marché, avec les IV obtenus en appliquant la même méthode numérique au prix NGARCH. En particulier, tabuler les erreurs de tarifications, dans le domaine de la IV, en fonction d'intervalles de maturité et de moneyness.

In [ ]:
# Simulation des returns avec le modèle GARCH pour plus grande maturité sous Q
S_unique    = np.unique(option_info.S_t)
date_unique = np.unique(option_info.date)
DTM_unique  = np.unique(option_info.DTM)

n_days96 = max(option_info[option_info.date == date_unique[0]].DTM)
n_days20 = max(option_info[option_info.date == date_unique[1]].DTM)

n_paths  = 100000 
ex_r_96  = measure( *ng1996.simulateQ(1, n_days96, n_paths, ng1996.Q_predict_h())).ex_r
ex_r_20  = measure( *ng2020.simulateQ(1, n_days20, n_paths, ng2020.Q_predict_h())).ex_r

option_info['R_j'] = pd.NA
warnings.simplefilter('ignore')
for dtm in DTM_unique :
    option_DTM_i  = option_info[option_info.DTM == dtm]
    for date in date_unique:
        option_DTM_ti = option_DTM_i[option_DTM_i.date == date]
        if not option_DTM_ti.empty:
            if pd.DatetimeIndex([date]).year == 1996:
                ex_r = ex_r_96
            else:
                ex_r = ex_r_20
            R_j    = np.exp(np.apply_along_axis(sum, 0, ex_r[:dtm]))
            for i in option_DTM_ti.index:
                option_info.R_j[i] = R_j

warnings.resetwarnings()

In [ ]:
option_info['Option_price'] = pd.NA

option_info_96 = option_info[option_info.date == date_unique[0]]
option_info_20 = option_info[option_info.date == date_unique[1]]

option_price_96 = [ng1996.option_price(info.R_j, 
                                       info.F_CBOE, 
                                       info.strike_price/1000,
                                       info.r_f,
                                       info.DTM,
                                       info.cp_flag == 'C') for _, info in option_info_96.iterrows()]
option_info.loc[option_info_96.index, 'Option_price'] = option_price_96

option_price_20 = [ng2020.option_price(info.R_j, 
                                       info.F_CBOE, 
                                       info.strike_price/1000,
                                       info.r_f,
                                       info.DTM,
                                       info.cp_flag == 'C') for _, info in option_info_20.iterrows()]
option_info.loc[option_info_20.index, 'Option_price'] = option_price_20

option_info[['date', 'exdate', 'cp_flag', 'strike_price', 'mean_bidask', 'impl_volatility', 'Option_price']]

In [ ]:
impl_vol_ng = bms.implied_volatility(opt_price = option_info.Option_price, 
                                     S         = option_info.exdiv_2, 
                                     K         = option_info.strike_price / 1000, 
                                     r         = option_info.r_f, 
                                     y         = 0, 
                                     T         = option_info.DTM / days_in_year, 
                                     is_call   = (option_info.cp_flag == 'C'))

option_info['IV_NGARCH'] = impl_vol_ng
option_info[['date','exdate','cp_flag', 'strike_price','impl_volatility','IV_method1','IV_method2','IV_NGARCH']]